### Odrive Setup

In [1]:
import odrive.enums
import numpy as np
from enum import Enum


class ErrorType(Enum):
    AXIS = 0
    MOTOR = 1
    SENSORLESS_ESTIMATOR = 2
    ENCODER = 3
    CONTROLLER = 4


error_type_infos = {
    ErrorType.MOTOR: ("motor_error", "Motor Errors"),
    ErrorType.AXIS: ("axis_error", "Axis Errors"),
    ErrorType.ENCODER: ("encoder_error", "Encoder Errors"),
}


def printErrors(error, error_type, prefix=""):
    error = int(error)
    if error_type == ErrorType.AXIS:
        error_map_prefix = "AXIS_ERROR_"
    elif error_type == ErrorType.MOTOR:
        error_map_prefix = "MOTOR_ERROR_"
    elif error_type == ErrorType.SENSORLESS_ESTIMATOR:
        error_map_prefix = "SENSORLESS_ESTIMATOR_ERROR_"
    elif error_type == ErrorType.ENCODER:
        error_map_prefix = "ENCODER_ERROR_"
    elif error_type == ErrorType.CONTROLLER:
        error_map_prefix = "CONTROLLER_ERROR_"
    error_map = {
        v: k for k, v in odrive.enums.__dict__.items() if k.startswith(error_map_prefix)
    }
    for bit in range(64):
        if error & (1 << bit) != 0:
            print(f"{prefix}{error_map.get(1 << bit)}")


def printErrorTimes(df, error_type):
    error_type_info = error_type_infos[error_type]
    idxs = np.where(df[error_type_info[0]] != df[error_type_info[0]].shift())

    for idx in idxs[0]:
        error_time_s = df.iloc[idx]["start_s"]
        error = df.iloc[idx][error_type_info[0]]
        print(f"{error_type_info[1]} ({error_time_s:.02f}):")
        if error == 0:
            print("  NONE")
        else:
            printErrors(error, error_type, "  ")

### Data Parsing Setup

In [2]:
import os
import pandas as pd

import plotly.graph_objects as go

# import plotly.express as px
# import plotly.subplots as subplots

csv_cols = [
    "dt_us",
    "voltage",
    "heartbeat",
    "wheel_rpm",
    "engine_rpm",
    "target_rpm",
    "velocity_command",
    "real_velocity_command",
    "shadow_count",
    "ignore1",
    "ignore2",
    "iq_measured",
    "flushed",
    "wheel_count",
    "engine_count",
    "iq_setpoint",
    "start_us",
    "stop_us",
    "current",
    "axis_error",
    "motor_error",
    "encoder_error",
]

wheel_diameter = 23
secondary_ratio = 12 / 6 * 45 / 17
wheel_ratio = 12 / 6 * 18 / 57
pitch_angle = 5
encoder_cpr = 8192

In [3]:
class DescribedDataframe(pd.DataFrame):
    _metadata = ["filename", "description"]

    @property
    def _constructor(self):
        return DescribedDataframe

In [4]:
def parseCSVFile(path):
    try:
        df = pd.read_csv(path, skiprows=1, header=None, names=csv_cols)
    except:
        return None
    return DescribedDataframe(df)


def parseBinaryFile(path):
    return None


def postProcessDataframe(df):
    df["start_s"] = df["start_us"] / 1e6

    df["secondary_rpm"] = df["wheel_rpm"] * secondary_ratio
    df["wheel_rpm"] = df["wheel_rpm"] * wheel_ratio
    df["wheel_mph"] = (df["wheel_rpm"] * wheel_diameter * np.pi) / (12 * 5280) * 60

    df["actuator_position_mm"] = -df["shadow_count"] / encoder_cpr * pitch_angle
    df["shift_ratio"] = df["secondary_rpm"] / df["engine_rpm"]

    motor_error_idxs = np.where(df["motor_error"] != 0)
    motor_error_bool = np.zeros(len(df))
    motor_error_bool[motor_error_idxs] = 1
    df["motor_error_bool"] = motor_error_bool


def addNormalizedColumns(df):
    for col in df:
        col_norm = f"norm_{col}"
        if col.startswith("norm_") or col_norm in df:
            continue
        col_obj = df[col]
        df[col_norm] = (col_obj - np.min(col_obj)) / (np.max(col_obj) - np.min(col_obj))

### Parse Data

In [5]:
def getTextLogs(log_dir):
    log_dir_contents = os.listdir(log_dir)
    txt_paths = []
    for potential_file in log_dir_contents:
        potential_file_path = os.path.join(log_dir, potential_file)
        potential_file_ext = os.path.splitext(potential_file)[1]
        if os.path.isfile(potential_file_path) and potential_file_ext == ".txt":
            txt_paths.append(potential_file_path)
    txt_paths.sort()
    return txt_paths


def filterFilesBySize(paths, size_kb):
    return [path for path in paths if os.path.getsize(path) >= size_kb * 1e3]

In [6]:
log_dir = "logs/"
generate_html = False
offline = False
show_figures = False
min_log_size_kb = 10

paths = getTextLogs(log_dir)
paths = filterFilesBySize(paths, min_log_size_kb)

In [7]:
dfs = [] 
# hehe troll

for path in paths:
    df = parseCSVFile(path)
    if df is None:
        continue
    postProcessDataframe(df)
    df.filename = os.path.basename(path)
    addNormalizedColumns(df)
    dfs.append(df)

### Figure Generators

In [8]:
def getRPMFigure(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["engine_rpm"], name="Engine RPM"))
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["target_rpm"], name="Target RPM"))
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="RPM",
        title=f"Engine RPM and Secondary RPM<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [9]:
def getRPMAndActuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_engine_rpm"], name="Engine RPM")
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["norm_actuator_position_mm"], name="Actuator Position"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Engine RPM, Secondary RPM, and Actuator Position<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [10]:
def getVehicleSpeedFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["wheel_mph"],
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Vehicle Speed (mph)",
        title=f"Vehicle Speed<br><sup>{df.filename}</sup>",
    )
    return fig

In [11]:
def getShiftRatioAndAcuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["norm_actuator_position_mm"],
            name="Actuator Position (mm)",
        )
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_shift_ratio"], name="Shift Ratio")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Actuator Position and Shift Ratio<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [12]:
def getVelocityCommandsFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["velocity_command"], name="Unclamped Velocity Command"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["real_velocity_command"], name="Velocity Command"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Velocity Command",
        title=f"Velocity Command<br><sup>{df.filename}</sup>",
    )
    return fig

In [13]:
def getShadowCountFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["shadow_count"], name="Shadow Count")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Shadow Count",
        title=f"Shadow Count<br><sup>{df.filename}</sup>",
    )
    return fig

In [14]:
def getEngineVsWheel(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["wheel_mph"], y=df["engine_rpm"], name="Engine RPM vs Wheel Speed"
        )
    )
    fig.update_layout(
        xaxis_title="Wheel Speed (mph)",
        yaxis_title="Engine RPM",
        title=f"Engine RPM vs Wheel Speed<br><sup>{df.filename}</sup>",
    )
    return fig


### Create Invidual Graphs (By Logs)

In [15]:
def figuresToHTML(figs, filename, offline=False):
    with open(filename, "w") as file:
        file.write("<html><head></head><body>" + "\n")
        for fig in figs:
            font_size_backup = fig.layout.font.size
            fig.update_layout(font={"size": 20})
            inner_html = (
                fig.to_html(include_plotlyjs=(True if offline else "cdn"))
                .split("<body>")[1]
                .split("</body>")[0]
            )
            file.write(inner_html)
            fig.update_layout(font={"size": font_size_backup})
        file.write("</body></html>" + "\n")

In [16]:
all_figs_by_log = []
for df in dfs:
    all_figs_by_log.append(
        [
            getRPMFigure(df),
            getRPMAndActuatorFigure(df),
            getVehicleSpeedFigure(df),
            getShiftRatioAndAcuatorFigure(df),
            getVelocityCommandsFigure(df),
            getShadowCountFigure(df),
            getEngineVsWheel(df),
        ]
    )

In [17]:
if generate_html:
    for path, figs_by_log in zip(paths, all_figs_by_log):
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{filename_without_ext}.html"
        figuresToHTML(figs_by_log, html_path, offline=offline)

In [18]:
import ipywidgets as widgets
from IPython.display import clear_output
#hehe troll 2
show_figures = True
if show_figures:
    dropdown = widgets.Dropdown(options=paths, value=paths[-1])

    def onDropdownChange(change):
        if change["type"] == "change" and change["name"] == "value":
            clear_output()
            idx = paths.index(change["new"])
            printErrorTimes(dfs[idx], ErrorType.MOTOR)
            display(dropdown)
            for fig in all_figs_by_log[idx]:
                fig.show()

    idx = -1
    dropdown.observe(onDropdownChange)
    printErrorTimes(dfs[idx], ErrorType.MOTOR)
    display(dropdown)
    for fig in all_figs_by_log[idx]:
        fig.show()

Motor Errors (5.61):
  NONE


Dropdown(index=1, options=('logs/log_2023-04-20_22-41-50.txt', 'logs/log_2023-04-20_23-00-20.txt'), value='log…

### Create Group Graphs (By Graph Type)

In [19]:
figure_names_and_funcs = [
    ("rpm", getRPMFigure),
    ("rpm-and-actuator", getRPMAndActuatorFigure),
    ("vehicle-speed", getVehicleSpeedFigure),
    ("shift-ratio-and-acuator", getShiftRatioAndAcuatorFigure),
    ("velocity-command", getVelocityCommandsFigure),
    ("shadow-count", getShadowCountFigure),
    ("engine-vs-wheel", getEngineVsWheel),
]
all_figs_by_type = []
for name, func in figure_names_and_funcs:
    figs_by_type = []
    for df in dfs:
        figs_by_type.append(func(df))
    all_figs_by_type.append((name, figs_by_type))

In [20]:
if generate_html:
    for figs_by_type in all_figs_by_type:
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{figs_by_type[0]}.html"
        figuresToHTML(figs_by_type[1], html_path, offline=offline)

### Odrive Helpers

### Error Logging

In [21]:
df = dfs[-2]
printErrorTimes(df, ErrorType.MOTOR)

Motor Errors (5.60):
  NONE
